In [51]:
#import packages
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import csv
#import pandasql as ps
import matplotlib.pyplot as plt
import shapefile as shp
import seaborn as sns
import random
import time
from sklearn.model_selection import train_test_split
#import pysal as ps
import scipy

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data

from torch.nn import functional
from torch.autograd import Variable

import statistics

#temporal pattern clustering
from tslearn.clustering import TimeSeriesKMeans, silhouette_score
from tslearn.utils import to_time_series_dataset
from sklearn import metrics

from dtaidistance import dtw
from dtaidistance import dtw_visualisation as dtwvis

from scipy.cluster.hierarchy import fcluster
from scipy.stats import chisquare
from scipy.stats import wasserstein_distance

#regression model
from patsy import dmatrices
import statsmodels.api as sm

from datetime import datetime

from pysal.model import spreg
from pysal.lib import weights
from pysal.explore import esda

In [307]:
path_ = 'G:/My Drive/2020/Bias/data-processing'
os.chdir(path_)

In [308]:
#get geoid with demo
#data processing
#demo and urban info for entire psrc
demo_reg = pd.read_csv('demo_reg_df.csv')

#obs info in Seattle
obs_Seattle = pd.read_csv('obs_Seattle.csv')

In [309]:
Seattle_geo = np.unique(obs_Seattle['GEOID10'])
demo_reg = demo_reg[demo_reg['GEOID10'].isin(Seattle_geo)]

geoids = []
for geoid in Seattle_geo:
    geoids.append(str(geoid))

In [310]:
#select spatial features
#Notice that downtown Seattle only has urban area
st_reg = demo_reg[['GEOID10', 'INTPTLAT10','INTPTLON10','total','Urban']]

#read the cb geo data
path_ = 'spatial_check/'
os.chdir(path_)

In [311]:
del(demo_reg)

In [52]:
Seattle_st_cb = gpd.read_file('Seattle_ct_spatial_features.shp')
Seattle_busstop_cb = gpd.read_file('transit_stop.shp')
Seattle_sidewalk_cb = gpd.read_file('Seattle_ct_spatial_features_sidewalks.shp')
Seattle_bus_cb = gpd.read_file('Seattle_ct_spatial_features_transit_rt.shp')

In [72]:
def get_seattle_zone(dt_file, geoids):
    return(dt_file[dt_file['GEOID10'].isin(geoids)])

In [80]:
Seattle_st_cb = get_seattle_zone(Seattle_st_cb, geoids)
Seattle_busstop_cb = get_seattle_zone(Seattle_busstop_cb, geoids)
Seattle_sidewalk_cb = get_seattle_zone(Seattle_sidewalk_cb, geoids)
Seattle_bus_cb = get_seattle_zone(Seattle_bus_cb, geoids)

Seattle_st_cb = Seattle_st_cb[['GEOID10', 'ARTCLASS', 'STREETTYPE', 'ARTDESCRIP']]
Seattle_busstop_cb = Seattle_busstop_cb[['OBJECTID', 'STOP_ID', 'STOP_TYPE', 'GEOID10']]
Seattle_sidewalk_cb = Seattle_sidewalk_cb[['CONDITION','GEOID10']]
Seattle_bus_cb = Seattle_bus_cb[['ROUTE_NUM', 'GEOID10']]

geoids = sorted(geoids)
st_class = sorted(st_class)

In [334]:
np.unique(Seattle_st_cb['ARTDESCRIP'][Seattle_st_cb['ARTCLASS']==0])


array(['Not Designated'], dtype=object)

In [275]:
#data handling
def get_data(dt, by_col, geoids):
    a = dt.groupby(by=by_col).size().to_frame()
    a.reset_index(inplace=True)
    
    b = a.groupby(by=by_col[0]).sum()
    b.reset_index(inplace=True)
    
    col_class = np.unique(a[by_col[1]])

    num_st_class = np.zeros((len(geoids), len(col_class)))

    geoid_row = {}
    para_col = {}

    index_ = 0
    for i in geoids:
        geoid_row[i] = index_
        index_ += 1
    index_ = 0
    for j in col_class:
        para_col[j] = index_
        index_ += 1

    for i in range(len(a.index)):
        row_ = geoid_row[a[by_col[0]].loc[i]]
        col_ = para_col[a[by_col[1]].loc[i]]
        val_ = a[0].loc[i]
        #print(row_, col_, val_, i)
        num_st_class[row_][col_] = val_
        
    col_name = list(para_col.keys())
    col_name.append(by_col[1]+'_sum')
        
    return(np.c_[(num_st_class), b[0].values], col_name)

#street data
st_dt, st_name = get_data(Seattle_st_cb, ['GEOID10','ARTCLASS'], geoids)
#bus stop
bus_dt, bus_name = get_data(Seattle_busstop_cb, ['GEOID10', 'STOP_TYPE'], geoids)
#sidewalk
walk_dt, walk_name = get_data(Seattle_sidewalk_cb, ['GEOID10', 'CONDITION'], geoids)
#bus route
a = Seattle_bus_cb.groupby(by=['GEOID10','ROUTE_NUM']).size().to_frame()
a.reset_index(inplace=True)
a['ROUTE_NUM'] = a['ROUTE_NUM'].astype(str)

num_route = []
for geoid in geoids:
    num_route.append(len(np.unique(a['ROUTE_NUM'][a['GEOID10'] == geoid].values)))

In [271]:
#st_reg

In [276]:
#obs_Seattle
def add_col(ori_dt, add_dt, col_name):
    a = ori_dt.copy()
    for i in range(len(col_name)):
        a[col_name[i]] = add_dt[:, i]
    return(a)

In [279]:
test_dt = add_col(st_reg, st_dt, st_name)
test_dt = add_col(test_dt, bus_dt, bus_name)
test_dt = add_col(test_dt, walk_dt, walk_name)
test_dt['route_num'] = num_route

In [312]:
#handling data index
hr = [0] * 7
hr.extend([1]*4)
hr.extend([2]*5)
hr.extend([3]*4)
hr.extend([0]*4)

set_hr_index = lambda x: hr[x]
obs_Seattle['hr_index'] = obs_Seattle['hr'].apply(set_hr_index)

In [318]:
obs_hr = obs_Seattle[['GEOID10', 'hr_index', 'obs']].groupby(by=['GEOID10', 'hr_index']).sum()
obs_hr.reset_index(inplace=True)

In [325]:
path = 'G:/My Drive/2021/Bias/'

In [319]:
obs_hr.to_csv(path + 'obs_hr.csv')

In [322]:
obs_hr.set_index('GEOID10', inplace=True)
test_dt.set_index('GEOID10', inplace=True)

merge_df = pd.merge(obs_hr, test_dt, left_index=True, right_index=True, how="left")

In [326]:
merge_df.to_csv(path+'spatial_temp_reg.csv')